In [1]:
! pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
!pip install transformers[torch]

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import pipeline

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

pipe = pipeline("text-generation", model="gpt2")

def get_response(prompt: str) -> None:
    """
    Generate a response from the GPT2 model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = pipe(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256,
        truncation=True,
    )
    print("Chatbot:", sequences[0]['generated_text'])

if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["lyrics"], max_length=40, padding="max_length", truncation=True)
    # Shift input ids to the right to create labels
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
songs = pd.read_csv("combined_songs.csv")
train_data, test_data = train_test_split(songs, test_size = 0.2, random_state = 69)
songs_split = np.array_split(train_data, 8)
def data_loading(songs):
    train_data_dict = {'lyrics': [str(ele["lyrics"]) if not pd.isnull(ele["lyrics"]) else '' for _, ele in songs.iterrows()],
                       'genre': [str(ele["genre"]) if not pd.isnull(ele["genre"]) else '' for _, ele in songs.iterrows()]}
    train_data = Dataset.from_dict(train_data_dict)

    # test_data_dict = {'lyrics': [str(ele["lyrics"]) if not pd.isnull(ele["lyrics"]) else '' for _, ele in test_data.iterrows()]}
    # test_data = Dataset.from_dict(test_data_dict)

    lm_train_data = train_data.map(tokenize_function, batched=True, num_proc=4, remove_columns=["lyrics","genre"])
    # lm_test_data = test_data.map(tokenize_function, batched=True, num_proc=4, remove_columns=["lyrics"])

    train_valid_data = lm_train_data.train_test_split(test_size=0.2)
    lm_train_data = train_valid_data["train"]
    lm_valid_data = train_valid_data["test"]
    return lm_train_data, lm_valid_data

In [7]:
test_data_dict = {'lyrics': [str(ele["lyrics"]) if not pd.isnull(ele["lyrics"]) else '' for _, ele in test_data.iterrows()],
                  'genre': [str(ele["genre"]) if not pd.isnull(ele["genre"]) else '' for _, ele in test_data.iterrows()]}
test_data = Dataset.from_dict(test_data_dict)

lm_test_data = test_data.map(tokenize_function, batched=True, num_proc=4, remove_columns=["lyrics","genre"])

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/16857 [00:00<?, ? examples/s]

In [8]:
training_args = TrainingArguments(
    "GPT2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01
)

for song in songs_split:
    try:
      model = GPT2LMHeadModel.from_pretrained("task1")
    except:
      model = model

    lm_train_data, lm_valid_data = data_loading(song)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=lm_train_data,
        eval_dataset=lm_valid_data,
    )

    trainer.train()

    trainer.save_model("task1")

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/8429 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,4.077200,3.827723
2,3.788700,3.805456
3,3.707100,3.804838


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/8429 [00:00<?, ? examples/s]

...

Map (num_proc=4):   0%|          | 0/8428 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,3.801900,3.673968
2,3.713400,3.672477
3,3.599200,3.675376


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/8428 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,3.758600,3.637899
2,3.664100,3.636170
3,3.564900,3.638328


In [11]:
import math
eval_results = trainer.evaluate(lm_test_data)
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 39.27


In [21]:
from huggingface_hub import notebook_login

notebook_login()

In [22]:
model.push_to_hub("guna-2222/NLP_task1")
tokenizer.push_to_hub("guna-2222/NLP_task1")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/guna-2222/NLP_task1/commit/ea879a399308764ddf627a395f3a5072c3432cda', commit_message='Upload tokenizer', commit_description='', oid='ea879a399308764ddf627a395f3a5072c3432cda', pr_url=None, pr_revision=None, pr_num=None)